In [4]:
from pyspark.sql import SparkSession

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.1.153:7077") \
        .appName("A2")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [27]:


#Import documents in English and Swedish: 
europarl_en= spark_context.textFile("hdfs://192.168.1.153:9000/europarl/europarl-v7.sv-en.en")
europarl_sv= spark_context.textFile("hdfs://192.168.1.153:9000/europarl/europarl-v7.sv-en.sv")



In [50]:
#Question A.1 Check the number of lines:

def split_line(line):
    return line.split()

lines_splitted_en = europarl_en.map(split_line);
lines_splitted_sv = europarl_sv.map(split_line);

number_of_lines_en=lines_splitted_en.count();
number_of_lines_sv=lines_splitted_sv.count();
partitions_en=europarl_en.getNumPartitions();
partitions_sv=europarl_sv.getNumPartitions();

print("Number of lines in English:",number_of_lines_en)
print("Number of lines in Swedish:",number_of_lines_sv)

print("Partitions English:",partitions_en,"Partitions Swedish:",partitions_sv)

Number of lines in English: 1862234
Number of lines in Swedish: 1862234
Partitions English: 2 Partitions Swedish: 3


2 3


In [45]:
#Question A.2

def Func(lines):
      lines = lines.lower()
      lines = lines.split(' ')
      return lines
rdd_en = europarl_en.map(Func)
rdd_sv = europarl_sv.map(Func)


In [51]:
#Verify number of lines and inspecto 10 entries from each RDD.

test_number_of_lines_en=rdd_en.count();
test_number_of_lines_sv=rdd_sv.count();
test_lowercase_en=rdd_en.take(10);
test_lowecase_sv=rdd_sv.take(10);

print("Number of lines in English:",test_number_of_lines_en)
print("Number of lines in Swedish:",test_number_of_lines_sv)

Number of lines in English: 1862234
Number of lines in Swedish: 1862234


In [88]:
#Question A.3

def get_tld(content):
    match = p.search(content)
    if match is not None:
        return match.group(1)
    else:
        return None

#words_by_tld_rdd = rdd_en.flatMapValues(lambda words: words.split(' '))\
#.flatMapValues(lambda words: words.split('\n'))\
#.mapValues(lambda word: word.strip())

counts = rdd_en.flatMap(lambda x: x.split(' ')) \
              .map(lambda x: (x, 1)) \
              .reduceByKey(add)
output = counts.collect()
for (word, count) in output:
    print("%s: %i" % (word, count))





NameError: name 'add' is not defined

In [ ]:
spark_context.stop()